# People Counts per Day

#### Counts computed by using computer vision algorithms to analyse when people are recorded walking past a predetermined line in our CCTV video feeds    

In [ ]:
from uo_bar_charts import BarChart
from IPython.display import Image,display,HTML
import os


chart = BarChart('people')
groups = {'Total People Count':[
'PER_PEOPLE_BLACKETT-BOOTS',
'PER_PEOPLE_THE_CORE_LINE_0',
'PER_PEOPLE_THE_CORE_LINE_1',
'PER_PEOPLE_BLACKETT-NORTHUMBERLAND-W',
'PER_PEOPLE_USB_LINE_0',
'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_6',
'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_3',
'PER_PEOPLE_NORTHUMERLAND_LINE_LONG_DISTANCE_HEAD_1',
'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_5',
'PER_PEOPLE_NORTHUMERLAND_LINE_LONG_DISTANCE_HEAD_0',
'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_2',
'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_0',
'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_1',
'PER_PEOPLE_NORTHUMERLAND_LINE_SHORT_DISTANCE_HEAD_4',
'PER_PEOPLE_NORTHUMERLAND_LINE_MID_DISTANCE_HEAD_0',
    ]}

for group,chart in chart.plot_data_group(groups,y_axis='People Count'):
    display(Image(chart))



